In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import ast
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
# Opening X-Wines dataset
wines   = pd.read_csv("XWines_Slim_1K_wines_150K_ratings\XWines_Slim_1K_wines.csv", low_memory=False, encoding="utf-8", memory_map=True)
ratings = pd.read_csv("XWines_Slim_1K_wines_150K_ratings\XWines_Slim_150K_ratings.csv", low_memory=False, encoding="utf-8", memory_map=True)
len(wines), len(ratings)

(1007, 150000)

In [3]:
print("Total wines:", wines.WineID.nunique(), "from", wines.Code.nunique() ,"different countries")
print("Total users:", ratings.UserID.nunique(), "who rated the wines")
print(f"             in the period between {ratings.Date.min()} and {ratings.Date.max()}")
print("Total ratings:", len(ratings), f"5-stars on wine vintages since {ratings.Vintage.min()}, including non-vintage 'N.V.'")

Total wines: 1007 from 31 different countries
Total users: 10561 who rated the wines
             in the period between 2012-04-19 20:46:00 and 2021-12-31 23:58:52
Total ratings: 150000 5-stars on wine vintages since 1950, including non-vintage 'N.V.'


In [4]:
wines_ratings = pd.merge(wines, ratings, on="WineID")
wines_ratings.columns

Index(['WineID', 'WineName', 'Type', 'Elaborate', 'Grapes', 'Harmonize', 'ABV',
       'Body', 'Acidity', 'Code', 'Country', 'RegionID', 'RegionName',
       'WineryID', 'WineryName', 'Website', 'Vintages', 'RatingID', 'UserID',
       'Vintage', 'Rating', 'Date'],
      dtype='object')

In [26]:
wines.info()
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1007 entries, 0 to 1006
Data columns (total 60 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   WineID            1007 non-null   int64  
 1   WineName          1007 non-null   object 
 2   Type              1007 non-null   object 
 3   Elaborate         1007 non-null   object 
 4   Grapes            1007 non-null   object 
 5   ABV               1007 non-null   float64
 6   Body              1007 non-null   object 
 7   Acidity           1007 non-null   object 
 8   Code              1007 non-null   object 
 9   Country           1007 non-null   object 
 10  RegionID          1007 non-null   int64  
 11  RegionName        1007 non-null   object 
 12  WineryID          1007 non-null   int64  
 13  WineryName        1007 non-null   object 
 14  Aperitif          1007 non-null   int32  
 15  Appetizer         1007 non-null   int32  
 16  Barbecue          1007 non-null   int32  


In [5]:
wines_ratings.info()
wines_ratings.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 22 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   WineID      150000 non-null  int64  
 1   WineName    150000 non-null  object 
 2   Type        150000 non-null  object 
 3   Elaborate   150000 non-null  object 
 4   Grapes      150000 non-null  object 
 5   Harmonize   150000 non-null  object 
 6   ABV         150000 non-null  float64
 7   Body        150000 non-null  object 
 8   Acidity     150000 non-null  object 
 9   Code        150000 non-null  object 
 10  Country     150000 non-null  object 
 11  RegionID    150000 non-null  int64  
 12  RegionName  150000 non-null  object 
 13  WineryID    150000 non-null  int64  
 14  WineryName  150000 non-null  object 
 15  Website     139935 non-null  object 
 16  Vintages    150000 non-null  object 
 17  RatingID    150000 non-null  int64  
 18  UserID      150000 non-null  int64  
 19  Vi

,WineID,WineName,Type,Elaborate,Grapes,Harmonize,ABV,Body,Acidity,Code,...,RegionName,WineryID,WineryName,Website,Vintages,RatingID,UserID,Vintage,Rating,Date
11400,101567,Tinto,Red,Assemblage/Blend,"['Castelão', 'Tinta Miúda', 'Camarate', 'Touri...","['Beef', 'Pasta', 'Veal', 'Poultry']",13.0,Full-bodied,High,PT,...,Lisboa,11644,Quinta de Bons-Ventos,NaN,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201...",19206191,1027352,2019,4.0,2021-05-12 23:23:08
85109,162497,Cabernet Sauvignon,Red,Varietal/100%,['Cabernet Sauvignon'],"['Beef', 'Lamb', 'Game Meat', 'Poultry']",14.5,Full-bodied,High,CL,...,Puente Alto,40375,Don Melchor,http://www.donmelchor.com,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201...",1574206,1165465,2008,5.0,2019-12-17 19:29:26
23885,106681,Pinot Noir Rosè,Rosé,Varietal/100%,['Pinot Noir'],"['Veal', 'Pork', 'Game Meat', 'Rich Fish', 'Po...",12.0,Medium-bodied,High,DE,...,Rheingau,15066,Leitz,http://www.leitz-wein.de,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201...",19643491,1401845,2020,3.5,2021-03-20 16:00:51
15753,101617,Cedro do Noval,Red,Assemblage/Blend,"['Syrah/Shiraz', 'Touriga Nacional', 'Touriga ...","['Beef', 'Pasta', 'Lamb', 'Game Meat']",14.0,Very full-bodied,High,PT,...,Duriense,10734,Quinta do Noval,http://www.quintadonoval.com,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201...",8728146,1223494,2014,4.0,2018-06-17 18:34:33
53368,111932,Margaux (Grand Cru Classé),Red,Varietal/100%,['Cabernet Sauvignon'],"['Beef', 'Lamb', 'Game Meat', 'Poultry']",14.0,Full-bodied,High,FR,...,Margaux,27980,Château Marquis d'Alesme,http://www.chateau-marquis-dalesme.fr,"[2020, 2019, 2018, 2017, 2016, 2015, 2014, 201...",4251424,1225418,2011,4.5,2021-07-02 17:48:22


In [6]:
columns_to_drop = ['Website', 'Vintages', 'Vintage']
wines_ratings = wines_ratings.drop(columns=columns_to_drop)

In [7]:
wines_ratings.sample(n=5)

,WineID,WineName,Type,Elaborate,Grapes,Harmonize,ABV,Body,Acidity,Code,Country,RegionID,RegionName,WineryID,WineryName,RatingID,UserID,Rating,Date
6785,100092,Nature,Sparkling,Assemblage/Blend,"['Chardonnay', 'Pinot Noir']","['Pork', 'Rich Fish', 'Shellfish']",12.5,Medium-bodied,High,BR,Brazil,1001,Serra Gaúcha,10412,Cave Geisse,2205215,1393244,3.5,2015-03-25 05:54:52
18268,101693,Douro,Red,Assemblage/Portuguese Red Blend,"['Touriga Nacional', 'Touriga Franca', 'Tinta ...","['Beef', 'Pasta', 'Lamb', 'Game Meat']",14.5,Very full-bodied,High,PT,Portugal,1032,Douro,11074,Quinta do Vale Meão,4329077,1390612,3.0,2017-10-29 02:07:22
49408,111679,Côtes De Gascogne Classic,White,Assemblage/Blend,"['Ugni Blanc', 'Gros Manseng', 'Sauvignon Blan...","['Rich Fish', 'Shellfish', 'Vegetarian']",10.5,Medium-bodied,Medium,FR,France,1265,Côtes de Gascogne,24972,Domaine du Tariquet,18416580,1220651,3.5,2021-01-21 19:54:35
143090,193483,Private Bin Sauvignon Blanc,White,Varietal/100%,['Sauvignon Blanc'],"['Shellfish', 'Vegetarian', 'Goat Cheese']",12.5,Light-bodied,High,NZ,New Zealand,2485,Marlborough,68930,Villa Maria,8629480,2053688,4.0,2016-01-08 20:21:24
68400,135871,Barolo Marcenasco,Red,Varietal/100%,['Nebbiolo'],"['Beef', 'Pasta', 'Lamb', 'Game Meat']",14.0,Very full-bodied,High,IT,Italy,1519,Barolo,41219,Renato Ratti,10988135,1168418,4.0,2019-08-14 16:58:10


In [8]:
print(type(wines_ratings['Grapes'].iloc[83662]))
print(wines_ratings['Grapes'].iloc[83662])

<class 'str'>
['Tempranillo', 'Garnacha Tinta']


In [9]:
def clean_grapes_column(grape_entry):
    try:
        # Try converting the string into a list using ast.literal_eval
        grape_list = ast.literal_eval(grape_entry)
        if isinstance(grape_list, list):
            # Join the list elements into a single string
            return " ".join(grape_list)
    except (ValueError, SyntaxError):
        return grape_entry

# Apply the function to the 'Grapes' column
wines_ratings['Grapes'] = wines_ratings['Grapes'].apply(clean_grapes_column)

wines_ratings.sample(5)


,WineID,WineName,Type,Elaborate,Grapes,Harmonize,ABV,Body,Acidity,Code,Country,RegionID,RegionName,WineryID,WineryName,RatingID,UserID,Rating,Date
71501,136180,Prosecco,Sparkling,Varietal/100%,Glera/Prosecco,"['Shellfish', 'Vegetarian', 'Appetizer', 'Snac...",11.0,Very light-bodied,High,IT,Italy,1659,Prosecco,41958,Ruffino,20810852,1030497,3.5,2021-07-29 05:58:51
133497,179060,Midnight Dark Red Blend,Red,Assemblage/Blend,Merlot Cabernet Sauvignon Petit Verdot Petite ...,"['Beef', 'Lamb', 'Veal', 'Poultry']",13.7,Full-bodied,High,US,United States,1763,California,54292,Ménage à Trois,14893141,1139690,4.0,2020-02-16 02:42:07
26310,111398,Rouge,Red,Varietal/100%,Grenache,"['Beef', 'Lamb', 'Game Meat']",13.0,Full-bodied,High,FR,France,1421,Rhone Valley,30896,La Vieille Ferme,10857169,1108527,3.5,2017-07-10 02:00:08
132120,179054,Cabernet Sauvignon,Red,Varietal/100%,Cabernet Sauvignon,"['Beef', 'Lamb', 'Game Meat', 'Poultry']",13.8,Full-bodied,High,US,United States,1751,Alexander Valley,59190,Jordan,12153571,1061957,4.0,2021-12-07 14:10:01
67550,135867,Le Difese,Red,Assemblage/Blend,Cabernet Sauvignon Sangiovese,"['Beef', 'Lamb', 'Veal', 'Poultry', 'Cured Meat']",13.0,Medium-bodied,Medium,IT,Italy,1699,Toscana,49339,Tenuta San Guido,11197884,1204084,3.5,2019-05-12 08:00:52


In [10]:
# For Harmonize column
def clean_harmonize_column(harmonize_entry):
    try:
        # Try converting the string into a list using ast.literal_eval
        harmonize_list = ast.literal_eval(harmonize_entry)
        return harmonize_list
    except (ValueError, SyntaxError):
        return harmonize_entry

# Apply the function to the 'Harmonize' column
wines_ratings['Harmonize'] = wines_ratings['Harmonize'].apply(clean_harmonize_column)

wines_ratings.sample(5)

,WineID,WineName,Type,Elaborate,Grapes,Harmonize,ABV,Body,Acidity,Code,Country,RegionID,RegionName,WineryID,WineryName,RatingID,UserID,Rating,Date
81158,155375,Cava Gran Reserva Brut Nature Reserva de la Fa...,Sparkling,Assemblage/Blend,Macabeo Xarel-lo Parellada,"[Shellfish, Appetizer, Snack, Lean Fish, Cured...",12.0,Light-bodied,High,ES,Spain,2005,Penedès,36672,Juvé & Camps,5387319,1127087,3.5,2016-11-16 20:44:10
51769,111773,Lacoste Borie Pauillac,Red,Varietal/100%,Cabernet Franc,"[Beef, Lamb, Game Meat, Poultry]",13.0,Full-bodied,High,FR,France,1396,Pauillac,29282,Château Grand-Puy-Lacoste,5085992,1226307,5.0,2018-01-19 14:25:30
77098,155308,Rioja Gran Reserva 904,Red,Varietal/100%,Tempranillo,"[Beef, Lamb, Veal, Poultry]",13.2,Full-bodied,High,ES,Spain,2014,Rioja,37809,La Rioja Alta,672878,1035878,4.5,2017-02-16 19:19:20
65716,135860,Brunello di Montalcino,Red,Varietal/100%,Sangiovese,"[Beef, Lamb, Game Meat, Poultry]",14.5,Very full-bodied,High,IT,Italy,1531,Brunello di Montalcino,49416,Tenuta Il Poggione,636327,1180773,4.5,2020-07-18 14:01:30
73632,137981,Lambrusco dell'Emilia Rosso,Sparkling,Varietal/100%,Lambrusco,"[Pork, Rich Fish, Maturated Cheese, Hard Chees...",8.0,Medium-bodied,Medium,IT,Italy,1582,Emilia-Romagna,42862,Fratelli Cella,20961299,1356612,3.5,2018-12-22 00:21:21


In [19]:
# Create one-hot encoding for the 'Harmonize' column
mlb = MultiLabelBinarizer()
harmonize_encoded = pd.DataFrame(mlb.fit_transform(wines['Harmonize']), columns=mlb.classes_, index=wines.index)

# Concatenate the one-hot encoded dataframe back to the original wines_ratings dataframe
wines = pd.concat([wines, harmonize_encoded], axis=1)

# Drop the original 'Harmonize'
wines.drop(columns=['Harmonize'], inplace=True)

# Check the resulting dataframe
print(wines.head().to_markdown())


|    |   WineID | WineName                        | Type      | Elaborate                     | Grapes                    |   ABV | Body          | Acidity   | Code   | Country   |   RegionID | RegionName   |   WineryID | WineryName   |   Aperitif |   Appetizer |   Barbecue |   Beef |   Blue Cheese |   Cake |   Cheese |   Chicken |   Chocolate |   Codfish |   Cold Cuts |   Cream |   Cured Meat |   Dessert |   Duck |   Fish |   French Fries |   Fruit |   Fruit Dessert |   Game Meat |   Goat Cheese |   Grilled |   Ham |   Hard Cheese |   Lamb |   Lean Fish |   Light Stews |   Maturated Cheese |   Mushrooms |   Pasta |   Pizza |   Pork |   Poultry |   Rich Fish |   Risotto |   Salad |   Seafood |   Shellfish |   Snack |   Soft Cheese |   Soufflé |   Spicy Food |   Sweet Dessert |   Tomato Dishes |   Veal |   Vegetarian |
|---:|---------:|:--------------------------------|:----------|:------------------------------|:--------------------------|------:|:--------------|:----------|:-------|:-

In [12]:
# Save the DataFrame as a CSV file
wines_ratings.to_csv('wines_ratings_processed.csv', index=False)